This notebook is for developing my new approach to VCF filtering
Because the attributes of VCFs created with different callers vary
I've made a new filtering strategy that works with any set of arbitrary filters
the state

In [38]:
#PopGenVCF

import vcf, sys, re
from collections import Counter
from ness_vcf import SFS


def test_site(pyVCF_RecordObject, recordAttribute_filters, recordInfo_filters, lenient=True, verbose = False):
    """
    this will filter based on whole site categories
    (1)Attributes of the record that can be accessed like this "record.ATTRIBUTE"
        e.g. QUAL, POS, FILTER, REF, ALT,  num_het, num_called, is_indel, is_sv etc
    (2)INFO field filters accessed like this record.INFO[field[
        e.g. MQ, DP, AF, AN etc

    THE challenge with whole SITE filters is that they won't necessarily all be numbers:
        e.g. we may want a site with QUAL > 30 and MQ > 20 BUT with ReF
    We need the filter dict to include the name of the thing plus the test
    then use eval to do the test:
        eg. f = 'QUAL': '> 30.0'
            if not eval('pyVCF_RecordObject.%s %s' %(f, SiteProperty_filter_dict[f]))
            #which is thq equivalent of eval(record.QUAL > 30)
    """ 
    for f in recordAttribute_filters:
        try:
            if not eval('pyVCF_RecordObject.%s %s' %(f, recordAttribute_filters[f])): #this means that along the way if any of the filters fail its over for this call
                return False
        except AttributeError:
            if verbose: sys.stderr.write("%s does not exist in record at site %s:%i \n" %(f, pyVCF_RecordObject.CHROM, pyVCF_RecordObject.POS))
            if lenient: continue
            else: return False
    for f in recordInfo_filters:
        try:
            if not eval("pyVCF_RecordObject.INFO['%s'] %s" %(f, recordInfo_filters[f])): #this means that along the way if any of the filters fail its over for this call
                return False
        except KeyError:
            if verbose: sys.stderr.write("%s does not exist in the INFO field of site %s:%i \n" %(f, pyVCF_RecordObject.CHROM, pyVCF_RecordObject.POS))
            if lenient: continue
            else: return False
    #if a site got here without returning false its a keeper
    return True

            
def test_genotype(pyVCF_CallObject, callFormat_filters, callAttribute_filters, lenient=True, verbose =False):
    """
    we want this function to take a PyVCF call object and test whether it passes all the defined filters
    if that field doesn't exist in this call object either return fail or go onto the other filters
    filters = {'min_GQ': min_GQ, 'min_DP': min_DP, 'min_RGQ': min_RGQ, 'min_purity': min_purity, 'called': called, ' heterozygote': heterozygote}
    # ???how do we translate the filters you want to a series of tests ???
        so things in the genotype field will be stored in a dictionary
            eg call['GT'] = 0, call['RGQ'] = 99 #need to be sure these are all minimum values
            these things can be accessed using the filter name to get the value
        other filters that are tests can be accessed from pyVCFs built in convenience functions
            eg. call.is_snp, call.called - NEED TO BE CAREFUL to ensure these are all True/FALSE
            to access these we can use the filter name eg 'called'  or 'is_het' 
            along with the 'getattr(c, filter)' to return the value
    perhaps the best way to set this up is to pass two dicts:
        (1) GT call filters ie anything that might show up in the record field
        (2) pyVCF attributes 
    """
    #####################
    if not pyVCF_CallObject.called:
        return False
    for f in callFormat_filters:
        try:
            if not eval("pyVCF_CallObject['%s'] %s" %(f, callFormat_filters[f])): #this means that along the way if any of the filters fail its over for this call
               return False
        except AttributeError: #this catches cases where the field doesn't exist in that call - 
            if verbose: sys.stderr.write("%s does not exist in genotype %s at site %s:%i \n" %(f, pyVCF_CallObject.sample,c.site.CHROM, c.site.POS))
            if lenient : continue
            else: return False
    for f in callAttribute_filters:
        try:
            if not eval('pyVCF_CallObject.%s %s' %(f, callAttribute_filters[f])): #this means that along the way if any of the filters fail its over for this call
                return False
        except AttributeError: #this catches cases where the field doesn't exist in that call - 
            if verbose: sys.stderr.write("%s is not an attribute of genotype %s at site %s:%i \n" %(f, pyVCF_CallObject.sample,c.site.CHROM, c.site.POS))
            if lenient : continue
            else: return False  
    # if you've reached here without returning False the site must have passed
    return True

def genotypes_to_MAF(record, recordAttribute_filters=None, recordINFO_filters=None, callFormat_filters=None, callAttribute_filters=None, samples='all', site_lenient=True, call_lenient=True, ploidy=2, vcf_ploidy=2, minHQ_GTs =2 ):
	##########################################################################################
	""" takes record and returns minor allele freq and num_called which exceed quality threshold """
	##########################################################################################
	if record.is_monomorphic and test_site(record,recordAttribute_filters, recordINFO_filters, lenient = site_lenient):
		if samples == 'all' or samples == ['all']:
			GTs = [i['GT'] for i in record.samples if test_genotype(i,callFormat_filters, callAttribute_filters, lenient=call_lenient)]
		else:
			GTs = [record.genotype(i)['GT'] for i in samples if test_genotype(i,callFormat_filters, callAttribute_filters, lenient=call_lenient)]
		if len(GTs) > minHQ_GTs:
			return (0.0, len(GTs)*ploidy)
		else: return None
	elif test_site(record,recordAttribute_filters, recordINFO_filters, lenient = site_lenient) and not record.is_monomorphic:
		if samples == 'all' or samples == ['all']:
			GTs = [i['GT'] for i in record.samples if test_genotype(i,callFormat_filters, callAttribute_filters, lenient=call_lenient)]
		else:
			GTs = [record.genotype(i)['GT'] for i in samples if test_genotype(i,callFormat_filters, callAttribute_filters, lenient=call_lenient)]
		# now make some calculations
		try:
			if len(GTs) < minHQ_GTs: return None
			elif ploidy == 2: 
				MAF = Counter(re.findall('[0-9]', str(GTs))).most_common(2)[-1][-1]/(2.0*len(GTs))
				return (MAF, 2*len(GTs))
			elif ploidy == 1 and vcf_ploidy==2:
				MAF = Counter(re.findall('[0-9]', str(GTs))).most_common(2)[-1][-1]/(2.0*len(GTs))
				return (MAF, len(GTs))
			elif ploidy==1 and vcf_ploidy==1:
				MAF = Counter(re.findall('[0-9]', str(GTs))).most_common(2)[-1][-1]/(1.0*len(GTs))
				return (MAF, len(GTs))
			else: return None
		except ZeroDivisionError:
			return None
	else:
		return None
	# #if the site is invariant we only use depth filters
	# elif 'MQ' in record.INFO and record.num_called >= min_called and record.INFO['MQ']>=min_MQ and not indel(record):
	# 	if samples == 'all' or samples == ['all']: callable = len([i for i in record.samples if (i and i['DP']>min_DP)])
	# 	else: callable = len([record.genotype(i)['DP'] for i in samples if (record.genotype(i)['DP']>min_DP)])
	# 	if callable >= min_called:
	# 		return (0.0, callable*ploidy)
	# elif record.num_called >= min_called and not indel(record):
	# 	if samples == 'all' or samples == ['all']: callable = len([i for i in record.samples if (i and i['DP']>min_DP)])
	# 	else: callable = len([record.genotype(i)['DP'] for i in samples if (record.genotype(i)['DP']>min_DP)])
	# 	if callable >= min_called:
	# 		return (0.0, callable*ploidy)
	# else: 
	# 	return None

def SFS_from_genotypes(vcf_reader, region, recordAttribute_filters=None, recordINFO_filters=None, callFormat_filters=None, callAttribute_filters=None, samples='all', site_lenient=True, call_lenient=True, ploidy=2, vcf_ploidy=2,minHQ_GTs=2):
	"""this goes to each site in a VCF and calculates the alternate allele frequency then summarises those calculations in SFS objects
	It returns a dictionary of SFSs - where the key is the number of alleles and the value is the SFS of sites with that many alleles"""
	chromosome = str(region.split(":")[0])
	start_coord = int(region.split(":")[1].split("-")[0])
	end_coord = int(region.split(":")[1].split("-")[1])
	AFs =[genotypes_to_MAF(record, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters, samples=samples, site_lenient=site_lenient, call_lenient=call_lenient, ploidy=ploidy, vcf_ploidy=vcf_ploidy,minHQ_GTs=minHQ_GTs) \
			for record in vcf_reader.fetch(chromosome, start_coord, end_coord) \
			if genotypes_to_MAF(record, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters, samples=samples, site_lenient=site_lenient, call_lenient=call_lenient, ploidy=ploidy, vcf_ploidy=vcf_ploidy,minHQ_GTs=minHQ_GTs) !=None]
	SFSs={}
	if samples == ['all'] or samples == 'all':
		for i in range(minHQ_GTs,ploidy*len(vcf_reader.samples)+1):
			SFSs[i] = SFS([0]*(i+1))
	else:
		for i in range(minHQ_GTs, ploidy*len(samples)+1):
			SFSs[i] = SFS([0]*(i+1))
	for a in AFs:
		num_alleles = a[1]
		freq = a[0]
		SFSs[num_alleles].add(freq, num_alleles)
	return SFSs #dictionary of SFSs for multiple allele depths


##### test a few things
# region = 'chromosome_1:1000000-1010000'
# vcf_file = '/Volumes/Ness_4TB/sequencing/chlamydomonas/quebec/all/haplotypeCaller/chromosome_1_4_development/chromosome_1.1000000-1100000.bpRES.vcf.gz'
# vr = vcf.Reader(filename=vcf_file)

# recordAttribute_filters={"num_called": ">8", "num_hets": "==0"}
# recordINFO_filters={}
# callFormat_filters={"GQ": ">30", "RGQ": ">30", "DP":">5"}
# callAttribute_filters={}
# SFSs = SFS_from_genotypes(vr, region, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters,  ploidy=1, vcf_ploidy=1)

# SFSs[10].summarise()


def vcf_stat_histo(vr, region, stat, bin_width, samples = 'all'):
    chromosome = str(region.split(":")[0])
    start_coord = int(region.split(":")[1].split("-")[0])
    end_coord = int(region.split(":")[1].split("-")[1])
    histo = OrderedDict()
    for r in vr.fetch(chromosome, start_coord, end_coord):
        r.stat
    AFs =[genotypes_to_MAF(record, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters, samples=samples, site_lenient=site_lenient, call_lenient=call_lenient, ploidy=ploidy, vcf_ploidy=vcf_ploidy,minHQ_GTs=minHQ_GTs) \


In [62]:
hapvcf = vcf.Reader(filename='/Volumes/Ness_4TB/sequencing/chlamydomonas/quebec/all/haplotypeCaller/chromosome_1_4_development/chromosome_1.1000000-1100000.vcf.gz')
#freevcf =vcf.Reader(filename='/Volumes/Ness_4TB/sequencing/chlamydomonas/quebec/all/freebayes/chromosome_1.dev/quebec_all.chromosome_1_1000000_1100000.vcf.gz') 

### This vcf is made with max-complex-gap ==0  which means that there shouldn't be any TYPE, snp,snp sites - ie more or less one site per line (except indels)
freevcf =vcf.Reader(filename='/Volumes/Ness_4TB/sequencing/chlamydomonas/quebec/all/freebayes/chromosome_1.dev/quebec_all.chromosome_1_1000000_1100000.freebayes.bpRES.vcf.gz') 

In [64]:
recordAttribute_filters={"num_called": ">8", "num_hets": "==0"}
recordINFO_filters={}
callFormat_filters={"DP":">5", "GQ" : ">0", "RGQ" : ">00"}
callAttribute_filters={}

In [65]:
freeSFSs = SFS_from_genotypes(freevcf, region, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters,  ploidy=1, vcf_ploidy=1)
for s in freeSFSs:
    freeSFSs[s].summarise(header=False)


False	False	0	0	0	None	None	None	[0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10	False	2	1	1	0.266666666667	0.17674288119	1.3026779048	[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
11	False	1	0	1	0.509090909091	0.341417152147	1.18559726194	[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
False	False	0	0	0	None	None	None	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
15	False	8	4	4	0.164285714286	0.153772233094	0.219180201508	[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3]
16	False	9	6	3	0.125	0.100455192819	0.7083234691

In [66]:
hapSFSs =  SFS_from_genotypes(hapvcf, region, recordAttribute_filters=recordAttribute_filters, recordINFO_filters=recordINFO_filters, callFormat_filters=callFormat_filters, callAttribute_filters=callAttribute_filters,  ploidy=1, vcf_ploidy=1)
for s in hapSFSs:
     hapSFSs[s].summarise(header=False)
 

False	False	0	0	0	None	None	None	[0, 0, 0]
3	False	18	18	0	0.0	0.0	0.0	[18, 0, 0, 0]
4	False	13	13	0	0.0	0.0	0.0	[13, 0, 0, 0, 0]
5	False	14	14	0	0.0	0.0	0.0	[14, 0, 0, 0, 0, 0]
6	False	7	7	0	0.0	0.0	0.0	[7, 0, 0, 0, 0, 0, 0]
7	False	15	14	1	0.0380952380952	0.0272108843537	1.3416407865	[14, 0, 0, 1, 0, 0, 0, 0]
8	False	12	10	2	0.0922619047619	0.0642791551882	1.62122726671	[10, 0, 0, 1, 1, 0, 0, 0, 0]
9	False	19	14	5	0.108187134503	0.0968255066049	0.497044016364	[14, 2, 0, 1, 2, 0, 0, 0, 0, 0]
10	False	18	16	2	0.0592592592593	0.0392761958199	1.64145266725	[16, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
11	False	28	26	2	0.0298701298701	0.0243869394391	0.687619159566	[26, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
12	False	45	43	2	0.0198653198653	0.014717301191	1.02214179656	[43, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
13	False	52	41	11	0.0710059171598	0.0681676120232	0.168305718872	[40, 2, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1]
14	False	117	101	16	0.04884004884	0.0430020078657	0.556986482607	[99, 3, 3, 1, 7, 0, 2, 0,

The strange result here is that @ first glance FREEBAYES has WAY fewer sites!!!
*HOWEVER*, this is because it reports little haplotype chunks. 
Using the parameter '--max-complex-gap' or '--haplotype-length' you can specify the max length of these chunks
So what happens is you might have 2-'max-complex-gap' sites all reported in a single VCF line
This is great that FreeBayes does this - but its fucking annoying to parse!!!



    freebayes -f $ref --ploidy 1 --max-complex-gap 0 --region chromosome_1:1000000-1010000 --theta 0.025 --genotype-qualities --report-monomorphic   /home2/data/sequencing/chlamydomonas/quebec/all/quebec_wt.realigned.bam >quebec_all.chromosome_1_1000000_1100000.freebayes.bpRES.vcf
  
  
# Eg of a weird SNP

    chromosome_1    1012696 .       TTGAATCCCCATGCGCCGGGGAGCCACGGTAGCCCGATG TTGAATCCCCATGTGCCGGGGAGCCACGGTAGCCCGATG,TTGAATCCCCATGCGCCGGGGAGCCACGGTAGCCCGATA 2383.31 .       AB=0,0;ABP=0,0;AC=1,4;AF=0.05,0.2;AN=20;AO=14,53;CIGAR=13M1X25M,38M1X;DP=280;DPB=427.923;DPRA=1,0.888889;EPP=12.937,3.05127;EPPR=3.42174;GTI=0;LEN=1,1;MEANALT=1,1.4;MQM=97.2857,98.4717;MQMR=97.9684;NS=20;NUMALT=2;ODDS=71.6637;PAIRED=1,1;PAIREDR=1;PAO=60.5,97.5;PQA=1774.17,3138.17;PQR=4459.67;PRO=141;QA=421,1541;QR=6232;RO=190;RPL=10,23;RPP=8.59409,5.01789;RPPR=3.42174;RPR=4,30;RUN=1,1;SAF=5,25;SAP=5.49198,3.37904;SAR=9,28;SRF=100;SRP=4.15318;SRR=90;TYPE=snp,snp;technology.illumina=1,1       GT:GQ:DP:RO:QR:AO:QA:GL 0:160.002:16:16:518:0,0:0,0:0,-56.7692,-56.0494 0:160.002:9:9:330:0,0:0,0:0,-34.3943,-34.231    0:160.002:11:9:277:0,0:0,0:0,-33.3755,-33.6034  0:160.002:10:9:266:0,1:0,34:0,-44.6318,-41.6855 0:160.002:15:15:490:0,0:0,0:0,-50.5683,-50.412  2:160.002:9:0:0:0,9:0,294:-42.6357,-42.4674,0   2:160.002:13:0:0:0,13:0,386:-54.3905,-54.1013,0 2:160.002:24:0:0:0,22:0,644:-65.0939,-64.6268,0 2:160.002:8:0:0:0,8:0,183:-32.912,-32.6298,0    0:160.002:21:18:616:0,0:0,0:0,-57.9422,-57.4051 1:160.002:14:0:0:14,0:421,0:-59.2619,0,-58.3784 0:160.002:19:18:588:0,0:0,0:0,-61.339,-60.7963  0:160.002:14:14:466:0,0:0,0:0,-45.5041,-45.1759 0:160.002:10:9:332:0,0:0,0:0,-31.4781,-31.7232  0:160.002:16:11:357:0,0:0,0:0,-39.6721,-39.8266 0:160.002:17:16:504:0,0:0,0:0,-48.5697,-47.9172 0:160.002:16:14:457:0,0:0,0:0,-43.4396,-43.4396 0:160.002:11:10:332:0,0:0,0:0,-32.5941,-32.0848 0:160.002:13:11:352:0,0:0,0:0,-38.467,-38.5823  0:160.002:14:11:347:0,0:0,0:0,-31.1668,-30.6165



```
- REF    TTGAATCCCCATG C GCCGGGGAGCCACGGTAGCCCGAT G
- ALT1   TTGAATCCCCATG T GCCGGGGAGCCACGGTAGCCCGAT G
- ALT2   TTGAATCCCCATG C GCCGGGGAGCCACGGTAGCCCGAT A
```

In [68]:

def mean_weightedBySites(SFSs, min_alleles=1):
    total_sites = 0.0
    total_pi = 0.0
    for s in SFSs:
        if s >min_alleles:
            try:
                total_pi += SFSs[s].sites() * SFSs[s].theta_pi()
                total_sites +=SFSs[s].sites()
            except TypeError:
                continue
    return total_pi/total_sites
    
print freeSFSs[20].theta_pi(), freeSFSs[20].taj_D()
print hapSFSs[18].theta_pi(), hapSFSs[18].taj_D()

print "Freebayes:",  mean_weightedBySites(freeSFSs, 8)
print "HaplotypeCaller:", mean_weightedBySites(hapSFSs,  8)

0.019540452475 -0.868736138935
0.0223430694959 -0.833414618249
Freebayes: 0.0169554122898
HaplotypeCaller: 0.0201226113439
